In [ ]:
import torch
import torchvision
from torchvision import datasets 
import torchvision.transforms  as T
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch import nn 
import torch.nn.functional as F


In [ ]:
Data_cifar = torchvision.datasets.CIFAR10 (
    
    root='./DATASET/', # donde queremos descargar 
    train=True, 
    #download=True,
    transform= T.Compose([T.ToTensor() , #T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ] ) #
) 

In [ ]:
# observamos el algunos ejemplos del set de datos 

plot = plt.figure(figsize=(8,8))
fila , columna = 3 ,3 

for i in range(1 , columna * fila + 1 ):
    pick = torch.randint(len(Data_cifar), size=(1,)).item()
    
    imgaen , label = Data_cifar[pick]
    
    plot.add_subplot(fila,columna,i)
    plt.title(str(label))
    plt.axis('off')
    plt.imshow(imgaen.permute(1, 2, 0))
plt.show()

In [ ]:
print(len(Data_cifar))

In [ ]:
##Para deividir el set de entrenamiento usaremos usaremos Prueba , validacion , entrenamiento = random_split() dentro de ( pondremos el data set y los procentajes  )


entrenamiento , validacion , prueba = torch.utils.data.random_split( Data_cifar , [0.8 , 0.1 , 0.1] )


print(len(entrenamiento))
print(len(validacion))
print(len(prueba))

train_loader = DataLoader( 
    dataset=entrenamiento,                          
    batch_size=512,                      
    shuffle=True                       
                          )

val_loader = DataLoader(
    dataset=validacion,
    batch_size=512,
    shuffle=True  
)

prueba_loader = DataLoader( 
        dataset=prueba,  
        batch_size=32,                            
        shuffle=True    
                            )



In [ ]:
class Redneuronal(nn.Module): 
    def __init__(self):
        super().__init__() 
       
        
        self.aplanar = nn.Flatten() 
        self.red = nn.Sequential( 
            
            nn.Linear(32*32*3 , 100), 
            nn.ReLU(), 
            nn.Linear(100,50), 
            nn.ReLU(),
            nn.Linear(50,10),
        )
        
        
    def forward(self , x ): 
        x = self.aplanar(x)
        output = self.red(x)

        return output


In [ ]:


class arquitectura_cnn(nn.Module): 

    def __init__(self , in_canal,canal1,canal2,canal3,canal4,canal5,in_FC, kernel_size):
        super().__init__() 
       
        
        self.capa1_conv =nn.Conv2d(in_channels=in_canal , out_channels=canal1 , kernel_size=kernel_size , padding=1)
        self.BN1 = nn.BatchNorm2d(num_features=canal1)
        self.capa2_conv = nn.Conv2d(in_channels=canal1 , out_channels=canal2 , kernel_size=kernel_size , padding=1)
        self.BN2 = nn.BatchNorm2d(num_features=canal2)
        self.capa3_conv = nn.Conv2d(in_channels=canal2 , out_channels=canal3 , kernel_size=kernel_size , padding=1) 
        self.maxpool = nn.MaxPool2d(2,2) 
        self.capa4_conv = nn.Conv2d(in_channels=canal3 , out_channels=canal4 , kernel_size=kernel_size , padding=1)
        self.BN3 = nn.BatchNorm2d(num_features=canal4)
        self.capa5_conv = nn.Conv2d(in_channels=canal4 , out_channels=canal5 , kernel_size=kernel_size , padding=1)
        self.flatten = nn.Flatten()
        self.fc_capa1 = nn.Linear ( in_features=in_FC, out_features=256) 
        self.out_capa = nn.Linear( in_features=256 , out_features= 10)
        
       
        
        
        
    def forward(self , x ): 

        y = F.relu(self.capa3_conv(F.relu(self.BN2(self.capa2_conv(F.relu(self.BN1(self.capa1_conv(x))))))))
        y = self.maxpool(y)
        y = F.relu(self.capa5_conv(F.relu(self.BN3(self.capa4_conv(y)))))
        y = self.maxpool(y)
        y = self.flatten(y)
        y = self.fc_capa1(y)
        prediciccion = self.out_capa(y)
        return prediciccion


In [ ]:
entrada = 3 
canal1 , canal2 , canal3 ,canal4 , canal5 , = 16 , 32 , 128 , 256 , 512

in_fc =  8*8*canal5

Modelo = arquitectura_cnn(entrada , canal1 , canal2 , canal3 ,canal4 , canal5, in_fc , 3 ).to('cuda')
Modelo

Parametros = sum(p.numel() for p in Modelo.parameters()) #p.numel() accede a la cantidad de perametros por capa si itera y se suma
print(f'La cantida de parametros es de  {Parametros}')
print(Modelo)

Hiperparametros 


In [ ]:
learning_rate = 0.001
Epochs = 10
fun_perdida = nn.CrossEntropyLoss()
optimizador = torch.optim.Adam(Modelo.parameters() , lr = learning_rate)

In [ ]:
def entrenamiento_loop(data_loader, modelo, fun_perdida, optimizador): 
    train_size = len(data_loader.dataset)
    nlotes = len(data_loader)
    modelo.train()
    
    perdida, accuracy = 0, 0
    
    for batch_idx, (x, y) in enumerate(data_loader):
        x, y = x.to("cuda"), y.to('cuda')
        # forward
        probabilidad = modelo(x)
        # backward
        loss = fun_perdida(probabilidad, y)
        loss.backward()
        optimizador.step()
        optimizador.zero_grad()
    
        perdida += loss.item()
        accuracy += (probabilidad.argmax(1) == y).type(torch.float).sum().item()
        
        # Corrección: mostrar progreso cada 10 lotes
        if batch_idx % 10 == 0: 
            current_loss = loss.item()
            processed_samples = (batch_idx + 1) * len(x)
            #print(f'\tPerdida: {current_loss:>7f} [{processed_samples:>5d}/{train_size:>5d}]')
    
    perdida /= nlotes
    accuracy /= train_size
    print(f'\t Exactitud/perdida promedio:')
    print(f'\t\t entrenamiento: {(100*accuracy):>0.2f}% / {perdida:>8f}')  # Aumentar precisión

In [ ]:
def val_loop(data_loader, modelo, fun_perdida): 
    val_size = len(data_loader.dataset)
    nlotes = len(data_loader)
    
    modelo.eval()
    
    perdida_val, accuracy = 0, 0
    
    with torch.no_grad():
        for X, y in data_loader: 
            X, y = X.to('cuda'), y.to('cuda')
            
            probabilidad = modelo(X)
            
            perdida_val += fun_perdida(probabilidad, y).item()
            accuracy += (probabilidad.argmax(1) == y).type(torch.float).sum().item()
            
    perdida_val /= nlotes
    accuracy /= val_size
    
    print(f'\t\tValidacion: {(100*accuracy):>0.2f}% / {perdida_val:>8f} \n')

In [ ]:
for i in range(Epochs):
    print(f'Epoca = {i+1} de {Epochs}')
    
    entrenamiento_loop(train_loader, Modelo, fun_perdida, optimizador)
    
    val_loop(val_loader, Modelo, fun_perdida)
    
print('listo')